In [36]:
import json
import requests

## Deploy the MCP Registry

### Deploy Catalog Data

Create additional `ConfigMap`s storing the MCP catalogs.

In [37]:
!oc create cm rh-catalog-mcp --from-file registry.json=../catalogs/rh-catalog-mcp.json
!oc create cm rh-partners-mcp --from-file registry.json=../catalogs/rh-partners-mcp.json
!oc create cm rh-one-mcp --from-file registry.json=../catalogs/rh-one-mcp.json

configmap/rh-catalog-mcp created
configmap/rh-partners-mcp created
configmap/rh-one-mcp created


### Deploy the Registry

Create the registry instance populated from the catalog sources.

In [38]:
!oc apply -f ../manifests/demo-registry.yaml
!oc wait pod -l app.kubernetes.io/name=demo-registry-api --for=condition=Ready --timeout=300s
!oc get pods

secret/demo-registry-db-password created
mcpregistry.toolhive.stacklok.dev/demo-registry created
pod/demo-registry-api-5d6bf9f84f-bgtq6 condition met
NAME                                       READY   STATUS    RESTARTS   AGE
cnpg-controller-manager-846579bb9d-cpkdk   1/1     Running   0          11d
demo-db-1                                  1/1     Running   0          76s
demo-registry-api-5d6bf9f84f-bgtq6         1/1     Running   0          11s
toolhive-operator-8d6c5fd48-8z6kv          1/1     Running   0          113s


Verify the spec contains 3 `configMapRef` and one `git` registries.

In [39]:
!oc get mcpregistry demo-registry -oyaml | yq '.spec.registries'

- configMapRef:
    key: registry.json
    name: rh-one-mcp
  format: toolhive
  name: rh-one-mcp
  syncPolicy:
    interval: 1h
- configMapRef:
    key: registry.json
    name: rh-catalog-mcp
  format: toolhive
  name: rh-catalog-mcp
  syncPolicy:
    interval: 1h
- configMapRef:
    key: registry.json
    name: rh-partners-mcp
  format: toolhive
  name: rh-partners-mcp
  syncPolicy:
    interval: 1h
- filter:
    tags:
      exclude:
        - experimental
      include:
        - database
        - filesystem
        - api
  format: toolhive
  git:
    branch: main
    path: pkg/registry/data/registry.json
    repository: https://github.com/stacklok/toolhive
  name: toolhive
  syncPolicy:
    interval: 1h


Verify the exposed service.

In [40]:
!oc get svc -l app.kubernetes.io/component=registry-api

NAME                TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
demo-registry-api   ClusterIP   172.30.86.117   <none>        8080/TCP   24s


## Validating the Registries

**Prerequisite**: Expose the registry API service at port 8888.

```bash
oc port-forward svc/demo-registry-api 8888:8080
```

In [41]:
REGISTRY_API_SVC_HOST = "localhost"
REGISTRY_API_SVC_PORT = 8888
REGISTRY_PATH = "registry"
EXTENSIONS_PATH = "extension"

In [ ]:
# Build the URL to query a given registry for a given path
# If `registry` is `None` or empty, the aggregated registry will be used
def registry_api_url(registry: str, path: str):
    if registry is None or registry == "":
        return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{REGISTRY_PATH}/v0.1/{path}"
    return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{REGISTRY_PATH}/{registry}/v0.1/{path}"


# Get the list of MCP servers
def get_mcp_servers(registry: str = None):
    """Get the list of MCP servers from the registry API"""
    url = registry_api_url(registry, "servers")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching servers: {e}")
        raise


# Build the URL to query an extension endpoint for a given registry
# If `registry` is `None` or empty, the aggregated registry will be used
def extension_api_url(registry: str, path: str):
    if registry is None or registry == "":
        return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{EXTENSIONS_PATH}/v0/{path}"
    return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{EXTENSIONS_PATH}/{registry}/v0/{path}"


# Get the list of MCP registries
def get_mcp_registries():
    """Get the list of MCP registries from the extension API"""
    url = extension_api_url(None, "registries")
    print(url)
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching registries: {e}")
        raise

In [43]:
# Print simple card list of the given servers
def print_server_cards(servers_list):
    """Print servers in a card layout format"""

    for idx, server_item in enumerate(servers_list, 1):
        server = server_item["server"]
        name = server.get("name", "N/A")

        # Extract package information (assuming first package)
        package = server.get("packages", [{}])[0] if server.get("packages") else {}
        registry_type = package.get("registryType", "N/A")
        identifier = package.get("identifier", "N/A")
        transport = package.get("transport", {}).get("type", "N/A")

        # Print card
        print(f"{'=' * 80}")
        print(f"Server #{idx}")
        print(f"{'-' * 80}")
        print(f"Name:       {name}")
        print(f"Transport:  {transport}")
        print(f"Registry:   {registry_type}")
        print(f"Identifier: {identifier}")

In [45]:
# Print global servers
servers = get_mcp_servers()["servers"]
print_server_cards(servers)

Server #1
--------------------------------------------------------------------------------
Name:       io.github.stacklok/agentops-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/agentops-mcp
Server #2
--------------------------------------------------------------------------------
Name:       io.github.stacklok/agentql-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/agentql-mcp
Server #3
--------------------------------------------------------------------------------
Name:       io.github.stacklok/alation-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/alation-ai-agent-sdk
Server #4
--------------------------------------------------------------------------------
Name:       io.github.stacklok/alibaba-cloud-ops-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/alibaba-cloud-ops-mcp-server
Server #5
----------------------------------------------------------------------------

In [46]:
# Print first 3 servers for each registry
registries = get_mcp_registries()
for registry in registries["registries"]:
    registryName = registry["name"]

    print(f"{'*' * 80}")
    print(f"Registry: {registryName} [{registry['sourceType']}]")
    print(f"{'*' * 80}")
    servers = get_mcp_servers(registryName)["servers"]
    print_server_cards(servers[:3])

http://localhost:8888/extension/v0/registries
********************************************************************************
Registry: default [kubernetes]
********************************************************************************
********************************************************************************
Registry: rh-catalog-mcp [file]
********************************************************************************
Server #1
--------------------------------------------------------------------------------
Name:       io.github.stacklok/agentops-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/agentops-mcp
Server #2
--------------------------------------------------------------------------------
Name:       io.github.stacklok/agentql-mcp-server
Transport:  stdio
Registry:   oci
Identifier: quay.io/mcp-servers/agentql-mcp
Server #3
--------------------------------------------------------------------------------
Name:       io.github.stacklok/alati

## Conclusion

We successfully deployed and validated the MCP Registry on OpenShift:

- ✅ Created 3 `ConfigMap`-based catalog sources (`rh-catalog-mcp`, `rh-partners-mcp`, `rh-one-mcp`)
- ✅ Deployed the `demo-registry` instance aggregating 4 registries (3 `ConfigMap`s + 1 Git-based)
- ✅ Verified the registry API service is accessible and functioning correctly
- ✅ Validated server discovery across multiple registry sources

The MCP Registry is now ready for discovering and deploying MCP servers. Proceed to the next notebook to deploy and discover MCP servers using this registry.